In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
weekend_Jan_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in weekend_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

25920

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 1080)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
x[:,:2]

matrix([[ 352345.58212337,  349365.8698338 ],
        [ 362347.24968738,  358920.06835082],
        [ 656135.70404203,  659239.85596047],
        [ 666770.66182925,  673110.47886094],
        [ 175713.13948471,  176057.02508629],
        [ 235679.51835631,  233766.20179   ],
        [ 432851.94024934,  435375.80488904],
        [ 428862.53520429,  429748.31783243],
        [ 383197.77744651,  385983.94394234],
        [ 394997.56567081,  389055.95486397],
        [ 385868.75458212,  399346.76289193],
        [ 333535.8401955 ,  332337.34755611],
        [ 100046.15481047,   99823.31056737],
        [  99792.47154631,  103649.59555173],
        [ 166183.42761702,  155502.73524167],
        [ 148183.81282739,  147046.25125201],
        [ 221276.13104043,  216012.37518491],
        [ 226625.39293639,  223939.54326923],
        [ 371649.50288845,  376225.11111807],
        [ 333177.28884753,  344099.70331262],
        [ 499745.10215935,  516255.86797819],
        [ 531704.51494075,  524323

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 962)

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-01, 7e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.21603627e+05  0.00000000e+00  4.40e+03 9.36e+02  1.00e+06     0s
   1  -1.00280898e+06 -1.86594052e+06  1.15e+03 2.60e+02  2.86e+05     0s
   2  -1.80734248e+06 -3.58777759e+06  5.85e+01 1.31e+01  4.60e+04     0s
   3  -2.67281635e+06 -3.05506927e+06  5.85e-05 1.24e-05  6.8

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[5.379004327106812e-10,
 120527.27729417074,
 170239.67765486252,
 2.406639171764905e-10,
 44855.98192400546,
 4.0442088877311266e-10,
 4.0622942333747903e-10,
 5.224702554007828e-10,
 2.7965118140423985e-11,
 25579.818774470405,
 2.505382391471878e-11,
 2.7965190109150757e-11,
 2.6156394061566086e-11,
 2.6332353938394434e-11,
 497252.2331064317,
 1.8035915691681162e-11,
 1.3549953148297682e-10,
 2.406600985712551e-10,
 85295.33886702935,
 4.0449783654409637e-10,
 4.040083670590914e-10,
 82910.4059844836,
 192413.8249148967,
 1.0547965601294011e-07,
 1.0774133969340655e-07,
 4.4383178621212817e-07,
 10930.265997106706,
 125993.19158298452,
 2.90644329964477e-07,
 1.8034794393816557e-11,
 120896.50521866423,
 3.0967713600315736e-10,
 84145.0217212894,
 2.0544587212010772e-08,
 60000.19098139206,
 2.2805936484826788e-07,
 1.8034799029915736e-11,
 23816.76447050306,
 3.096195182584605e-10,
 57740.750126667044,
 75757.78974742832,
 8164.941031938807,
 1.6084387723858635e-09,
 1.78359077496